<a href="https://colab.research.google.com/github/friedelj/AAI-510-TEAM-03/blob/main/JFriedel_Ethics_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JFriedel            Ethics Class                    Assignment 3     18Mar 25

In [ ]:
pip install aif360

In [ ]:
!pip install lightgbm

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, accuracy_score
from aif360.metrics import ClassificationMetric
from lightgbm import LGBMClassifier
from tqdm import tqdm
from aif360.algorithms.preprocessing import DisparateImpactRemover
import warnings
warnings.filterwarnings("ignore")

In [ ]:
print(os.getcwd())

In [ ]:
# Load data
data=pd.read_csv(r"C:\Users\josep\credit_card_default_531v.1.csv")

In [ ]:
# data preview
data.head()

In [ ]:
# structure and data types
data.info()

In [ ]:
# missing value check
missing_values = data.isnull().sum()
print("Missing values:\n", missing_values)

In [ ]:
# unique values in columns related to gender <- dataset uses 1 for male and 2 for female
print("Unique values in 'SEX':", data['SEX'].unique())

In [ ]:
# Rename only the target column for consistency
data.rename(columns={"default payment next month": "DEFAULT_NEXT_MONTH"}, inplace=True)

In [ ]:
#encode SEX column: Male = 1, Female = 0 (already numeric, we are just replacing 2 with 0)
data["SEX"] = data["SEX"].replace({2:0}) # Male = 1, Female = 0

In [ ]:
#EDUCATION: Map numbers to clear numeric categories for interpretability
#Graduate = 1, University = 2, High School = 3, Other = 4
data["EDUCATION"] = data["EDUCATION"].replace({
    0: 4, # Map 0 to 'Other'
    1: 1, # Graduate
    2: 2, # University
    3: 3, # High School
    4: 4, # Other
    5: 4, # Other
    6: 4  # Other
})

In [ ]:
#MARRIAGE: Map numbers to clear numeric catergories for interpretability
#Married = 1, Single =2, Other = 3
data["MARRIAGE"] = data["MARRIAGE"].replace({
    0: 3, # Map 0 to 'Other'
    1: 1, # Married
    2: 2, # Single
    3: 3  # Other
})

In [ ]:
columns_to_keep = ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "DEFAULT_NEXT_MONTH"]
data_subset = data[columns_to_keep]

In [ ]:
#mean default rate by age
age_default_prob = data.groupby("AGE")["DEFAULT_NEXT_MONTH"].mean()

#plotting the probability of default by age
plt.figure(figsize=(12, 6))
sns.lineplot(x=age_default_prob.index, y=age_default_prob.values, marker="o", linestyle="-", color="blue")
plt.title("Probability of Default by Age", fontsize=16)
plt.xlabel("Age", fontsize=14)
plt.ylabel("Probability of Default", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis="both", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# bin ages into deciles
age_bins = pd.qcut(data["AGE"], q=10, duplicates="drop")
data["Age_Group"] = age_bins

# get range labels for deciles
age_range_labels = [
    f"{int(interval.left)}-{int(interval.right)}"
    for interval in age_bins.cat.categories
]

# mapping these range labels back to the data
data["Age_Group"] = data["Age_Group"].cat.rename_categories(age_range_labels)

# calculate default probabilities for each age group
age_default_prob = (
    data.groupby("Age_Group", observed=True)["DEFAULT_NEXT_MONTH"]
    .mean()
    .reset_index()
    .rename(columns={"DEFAULT_NEXT_MONTH": "Default_Probability"})
)

# plotting default probabilities
plt.figure(figsize=(14, 7))
sns.barplot(
    data=age_default_prob,
    x="Age_Group",
    y="Default_Probability",
    palette="coolwarm",
    edgecolor="black",
)
plt.axhline(
    y=age_default_prob["Default_Probability"].mean(),
    color="red",
    linestyle="--",
    linewidth=1.5,
    label="Mean Default Probability",
)
plt.title("Probability of Default by Age Group (Deciles)", fontsize=16, fontweight="bold")
plt.xlabel("Age Range (Deciles)", fontsize=14)
plt.ylabel("Probability of Default", fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(rotation=12)
plt.legend(fontsize=12, loc="upper right")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
#define privileged (26-47) and underpriveledged groups (21-25, 48+)
data["Age_Group"] = data["AGE"].apply(
lambda x: "Priveledged (Ages 26-47)" if 26 <= x <= 47 else "Underprivedged (Ages 21-25, 48+)"
)

# calculate default probabilities for each group
age_group_default_prob = (
    data.groupby("Age_Group", observed=True)["DEFAULT_NEXT_MONTH"]
    .mean()
    .reset_index()
    .rename(columns={"DEFAULT_NEXT_MONTH": "Default_Probability"})
)

#plot default probsbilities with a regression line
plt.figure(figsize=(12, 7))
sns.barplot(
    data=age_group_default_prob,
    x="Age_Group",
    y="Default_Probability",
palette="coolwarm",
edgecolor="black"
)

sns.regplot(
    x=np.arange(len(age_group_default_prob)),
    y=age_group_default_prob["Default_Probability"],
    scatter=False, color="blue", label="Regression Line", ci=None
)

plt.axhline(
    y=age_group_default_prob["Default_Probability"].mean(),
    color="red",
    linestyle="--",
    linewidth=1.5,
    label="Mean Default Probability"
)

plt.title("Probability of Default by Age Group", fontsize=16, fontweight="bold")
plt.xlabel("Age Group", fontsize=14)
plt.ylabel("Probability of Default", fontsize=14)
plt.xticks(ticks=np.arange(len(age_group_default_prob)), labels=age_group_default_prob["Age_Group"], fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, loc="upper right")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# define Age_Group for priveleged and underpriveledged groups
youngest_quartile = data["AGE"].quantile(0.25)
oldest_quartile = data["AGE"].quantile(0.75)

data["Age_Group"] = data["AGE"].apply(
    lambda x: 1 if youngest_quartile < x <= oldest_quartile else 0
) # 1 for Privilaged  0 for Underprivilaged

# double checking the gender encoding
data["Gender_Label"] = data["SEX"].replace({1: "Male", 0: "Female"})

# underpriveledged group
underpriviledged_group = data[data["Age_Group"] == 0]

# default probabilities for males and females in the underpriviledged group
default_rates_underpriviledged = (
    underpriviledged_group.groupby("Gender_Label")["DEFAULT_NEXT_MONTH"].mean() * 100
)

#bar plot for default rates in the underpriviledged group xgender
plt.figure(figsize=(10, 6))
sns.barplot(
    x=default_rates_underpriviledged.index,
    y=default_rates_underpriviledged.values,
    palette=["orange", "green"],
    edgecolor="black"
)

plt.title("Default Rates for Underprivileged Group by Gender", fontsize=16, fontweight="bold")
plt.ylabel("Default Rate (%)", fontsize=14)
plt.xlabel("Gender", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

##QUANTIFYING DATASET BIAS

In [ ]:
# set seed
rand = 531
np.random.seed(rand)

# defining our target and features
y = data["DEFAULT_NEXT_MONTH"]
X = data.drop(["DEFAULT_NEXT_MONTH"], axis=1).copy()

# splitting into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=rand)

In [ ]:
# priviledged and underpriviledged Groups for Age
X_train["Age_Group"] = X_train["AGE"].apply(lambda x: 1 if 26 <= x <= 47 else 0)
X_test["Age_Group"] = X_test["AGE"].apply(lambda x: 1 if 26 <= x <= 47 else 0)

In [ ]:
# gender
X_train["Gender_Label"] = X_train["SEX"].replace({1: 1, 0: 0}) # female 0, male 1
X_test["Gender_Label"] = X_test["SEX"].replace({1: 1, 0: 0})

In [ ]:
# combie X_train and y_train for BinaryLabelDataset
train_ds = BinaryLabelDataset(
    df=X_train.join(y_train),
    label_names=["DEFAULT_NEXT_MONTH"],
    protected_attribute_names=["Age_Group", "Gender_Label"],
    favorable_label=0, # 0 did not default
    unfavorable_label=1 # 1 defaulted
)

In [ ]:
# defining our groups, remeber- priviledged is 26-47 and underpriviledged is 21-25 & 48+
privileged_groups = [{"Age_Group": 1}]
unprivileged_groups = [{"Age_Group": 0}]

In [ ]:
# then, compute our metrics for training data
metrics_train_ds = BinaryLabelDatasetMetric(
    train_ds,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [ ]:
# print results from training data
print("Training Data Metrics:")
print(f"Statistical Parity Difference (SPD): {metrics_train_ds.statistical_parity_difference():.4f}")
print(f"Disparate Impact (DI): {metrics_train_ds.disparate_impact():.4f}")
print(f"Smoothed Empirical Differential Fairness (SEDF): {metrics_train_ds.smoothed_empirical_differential_fairness():.4f}")

## QUANTIFYING MODEL BIAS

In [ ]:
lgb_params = {
    'learning_rate': 0.4,
    'reg_alpha': 21,
    'reg_lambda':1,
    'scale_pos_weight': 1.8
}

lgb_base_model = LGBMClassifier(random_seed=531, max_depth=6, num_leaves=33, **lgb_params)
lgb_base_model.fit(X_train, y_train)

In [ ]:
#evaluate model
y_pred_test = lgb_base_model.predict(X_test)
y_pred_prob_test = lgb_base_model.predict_proba(X_test)[:, 1]

In [ ]:
# performance metrics
accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
roc_auc = roc_auc_score(y_test, y_pred_prob_test)

In [ ]:
# print results
print(f"Accuracy (Test): {accuracy:.4f}")
print(f"Precision (Test): {precision:.4f}")
print(f"Recall (Test): {recall:.4f}")
print(f"F1-Score (Test): {f1:.4f}")
print(f"ROC-AUC (Test): {roc_auc:.4f}")

In [ ]:
#evaluate model
y_pred_train = lgb_base_model.predict(X_train)
y_pred_prob_train = lgb_base_model.predict_proba(X_train)[:, 1]

In [ ]:
# performance metrics
accuracy_train = accuracy_score(y_train, y_pred_train)
precision_train = precision_score(y_train, y_pred_train)
recall_train = recall_score(y_train, y_pred_train)
f1_train = f1_score(y_train, y_pred_train)
roc_auc_train = roc_auc_score(y_train, y_pred_prob_train)

In [ ]:
print(f"Accuracy (Train): {accuracy_train:.4f}")
print(f"Precision (Train): {precision_train:.4f}")
print(f"Recall (Train): {recall_train:.4f}")
print(f"F1-Score (Train): {f1_train:.4f}")
print(f"ROC-AUC (Train): {roc_auc_train:.4f}")

##FAIRNESS METRICS

In [ ]:
# passing test_ds into BinaryLabelDataset
test_ds = BinaryLabelDataset(
    df=X_test.join(y_test),
    label_names=["DEFAULT_NEXT_MONTH"],
    protected_attribute_names=["Age_Group", "Gender_Label"],
    favorable_label=0, # 0 no default
    unfavorable_label=1 # 1 defaulted
)

In [ ]:
# add predictions and scores to the test dataset for our fairness metrics
test_pred_ds = test_ds.copy(deepcopy=True)
test_pred_ds.labels = y_pred_test.reshape(-1, 1)
test_pred_ds.scores = y_pred_prob_test.reshape(-1, 1)

In [ ]:
# Compute Fairness Metrics
metrics_test_cls = ClassificationMetric(
    test_ds,
    test_pred_ds,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [ ]:
# Print Fairness Metrics
print("\nFairness Metrics on Test Data:")
print(f"Statistical Parity Difference (SPD): {metrics_test_cls.statistical_parity_difference():.4f}")
print(f"Disparate Impact (DI): {metrics_test_cls.disparate_impact():.4f}")
print(f"Smoothed Empiracal Differential Fairness (SEDF): {metrics_train_ds.smoothed_empirical_differential_fairness():.4f}")

## Mitigating Bias

# Reweighting Method

In [ ]:
# apply reweighing
reweighter = Reweighing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

reweighter.fit(train_ds)
train_rw_ds = reweighter.transform(train_ds)

In [ ]:
# metrics for the reweighted training dataset
metrics_train_rw_ds = BinaryLabelDatasetMetric(
    train_rw_ds,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [ ]:
# metrics for the reweighted training dataset
metrics_train_rw_ds = BinaryLabelDatasetMetric(
    train_rw_ds,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [ ]:
# metrics for the reweighted training dataset
metrics_train_rw_ds = BinaryLabelDatasetMetric(
    train_rw_ds,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

## Training LightGPM on the Reweighted Dataset

In [ ]:
# train LightGbM
lgb_rw_model = LGBMClassifier(
    random_state=531,
    max_depth=6,
    num_leaves=33,
    **lgb_params
)

In [ ]:
# use reweighted dataset's instance weights
lgb_rw_model.fit(
    X_train,
    y_train,
    sample_weight=train_rw_ds.instance_weights
)

In [ ]:
# evaluate performance metrics
y_pred_test_rw = lgb_rw_model.predict(X_test)
y_pred_prob_test_rw = lgb_rw_model.predict_proba(X_test)[:, 1]

In [ ]:
# Compute Performance Metrics
accuracy_rw = accuracy_score(y_test, y_pred_test_rw)
precision_rw = precision_score(y_test, y_pred_test_rw)
recall_rw = recall_score(y_test, y_pred_test_rw)
f1_rw = f1_score(y_test, y_pred_test_rw)
roc_auc_rw = roc_auc_score(y_test, y_pred_prob_test_rw)

In [ ]:
print("\nPerformance Metrics for Reweighted Model:")
print(f"Accuracy (Test): {accuracy_rw:.4f}")
print(f"Precision (Test): {precision_rw:.4f}")
print(f"Recall (Test): {recall_rw:.4f}")
print(f"F1-Score (Test): {f1_rw:.4f}")
print(f"ROC-AUC (Test): {roc_auc_rw:.4f}")

In [ ]:
# Compute Fairness Metrics & Add Predictions/Scors to Test DS
test_pred_rw_ds = test_ds.copy(deepcopy=True)
test_pred_rw_ds.labels = y_pred_test_rw.reshape(-1, 1)
test_pred_rw_ds.scores = y_pred_prob_test_rw.reshape(-1,1)

In [ ]:
# Fairness Metrics for the ReWeighted Model
metrics_test_rw_cls = ClassificationMetric(
    test_ds,
    test_pred_rw_ds,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [ ]:
# Print Fairness Metrics
print("\nFairness Metrics for Reweighted Model on Test Data:")
print(f"Statistical Parity Difference (SPD): {metrics_test_rw_cls.statistical_parity_difference():.4f}")
print(f"Disparate Impact (DI): {metrics_test_rw_cls.disparate_impact():.4f}")
print(f"Equal Opportunity Difference (EOD): {metrics_test_rw_cls.equal_opportunity_difference():.4f}")
print(f"Average Odds Difference (AOD): {metrics_test_rw_cls.average_odds_difference():.4f}")
print(f"Differential Fairness Bias Amplification (DFBA): {metrics_test_rw_cls.differential_fairness_bias_amplification():.4f}")

##DISPARATE IMPACT REMOVER (DIR)

In [ ]:
# define the levels of repair
levels = np.hstack([np.linspace(0., 0.1, 41), np.linspace(0.2, 1, 9)])
protected_index = train_ds.feature_names.index("Age_Group")

In [ ]:
# initialize variables to track the best repair level
di = np.array([])  # Collect Disparate Impact for all levels
train_dir_ds = None
test_dir_ds = None
X_train_dir = None
X_test_dir = None
lgb_dir_model = None

In [ ]:
!pip install BlackBoxAuditing

In [ ]:
import lightgbm as lgb

In [ ]:
# Loop through repair levels
for level in tqdm(levels):
    #apply DIR at the current repair level
    di_remover = DisparateImpactRemover(repair_level=level)
    train_dir_ds_i = di_remover.fit_transform(train_ds)
    test_dir_ds_i = di_remover.fit_transform(test_ds)

    #remove protected attribute from features
    X_train_dir_i = np.delete(train_dir_ds_i.features, protected_index, axis=1)
    X_test_dir_i = np.delete(test_dir_ds_i.features, protected_index, axis=1)

    #train LightGBM model on the repaired daataset
    lgb_dir_model_i = lgb.LGBMClassifier(
        random_state=rand, max_depth=5, num_leaves=33, verbose=-1, **lgb_params
    )
    lgb_dir_model_i.fit(X_train_dir_i, train_dir_ds_i.labels)

    # predict on the repaired dataset
    test_dir_ds_pred_i = test_dir_ds_i.copy()
    test_dir_ds_pred_i.labels = lgb_dir_model_i.predict(X_test_dir_i)

    # fairness metrics
    metrics_test_dir_ds = BinaryLabelDatasetMetric(
        test_dir_ds_pred_i,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )
    di_i = metrics_test_dir_ds.disparate_impact()

    # track and print DI for this level
    print(F"Repair Level: {level:.2f}, Disparate Impact: {di_i:.4f}")

    #update thebest results if this level is closest to DI=1
    if (di.shape[0] == 0) or (np.min(np.abs(di -1)) >= abs(di_i -1)):
        train_dir_ds = train_dir_ds_i
        test_dir_ds = test_dir_ds_i
        X_train_dir = X_train_dir_i
        X_test_dir = X_test_dir_i
        lgb_dir_model = lgb_dir_model_i

    # append to DI List
    di = np.append(di, di_i)

In [ ]:
# Plot Disparate Impact across repair levels
di = di[:len(levels)]

In [ ]:
plt.close(plt.gcf())

In [ ]:
# Use Seaborn style for better visuals
sns.set(style="whitegrid")

plt.figure(figsize=(11, 6))

In [ ]:
# Plot the Disparate Impact (DI) against repair levels
plt.plot(levels, di, marker="x", linestyle="-", color="green", label="Disparate Impact")

In [ ]:
# Add labels, title, and legend
plt.ylabel("Disparate Impact (DI)", fontsize=14)
plt.xlabel("Repair Level", fontsize=14)
plt.title("Disparate Impact Across Repair Levels", fontsize=16)
plt.legend(fontsize=12)

In [ ]:
# Show grid for readability
plt.grid(True)

In [ ]:
# Display the plot
plt.show()

In [ ]:
from IPython.display import display
display(plt.gcf())

In [ ]:
# print the best repair level and its corresponding disparate impact
best_level = levels[np.argmin(np.abs(di - 1))]
print(f"Best Repair Level: {best_level:.2f}")
print(f"Disparate Impact at Best Level: {di[np.argmin(np.abs(di - 1))]:.4f}")

In [ ]:
# predict on the modified test data (from DIR)
y_pred_test_dir = lgb_dir_model.predict(X_test_dir)
y_pred_prob_test_dir = lgb_dir_model.predict_proba(X_test_dir)[:, 1]

In [ ]:
# compute Performance Metrics on the test data
accuracy_test_dir = accuracy_score(y_test, y_pred_test_dir)
precision_test_dir = precision_score(y_test, y_pred_test_dir)
recall_test_dir = recall_score(y_test, y_pred_test_dir)
f1_test_dir = f1_score(y_test, y_pred_test_dir)
roc_auc_test_dir = roc_auc_score(y_test, y_pred_prob_test_dir)

In [ ]:
# Display Performance Metrics
print(f"Accuracy: {accuracy_test_dir:.4f}")
print(f"Precision: {precision_test_dir:.4f}")
print(f"Recall: {recall_test_dir:.4f}")
print(f"F1-Score: {f1_test_dir:.4f}")
print(f"ROC-AUC: {roc_auc_test_dir:.4f}")

In [ ]:
# predictions and scores to the BinaryLabelDataset for our Fairness Metrics
test_dir_ds_pred = test_dir_ds.copy(deepcopy=True)
test_dir_ds_pred.labels = y_pred_test_dir.reshape(-1, 1)
test_dir_ds_pred.scores = y_pred_prob_test_dir.reshape(-1, 1)

In [ ]:
# Fairness Metrics
metrics_test_dir_cls = ClassificationMetric(
    test_dir_ds,
    test_dir_ds_pred,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [ ]:
# Display Fairness Metrics
print("\nFairness Metrics on Test Data (DIR) Applied):")
print(f"Statistical Parity Difference (SPD): {metrics_test_dir_cls.statistical_parity_difference():.4f}")
print(f"Disparate Impact (DI): {metrics_test_dir_cls.disparate_impact():.4f}")
print(f"Equal Opportunity Difference (EOD): {metrics_test_dir_cls.equal_opportunity_difference():.4f}")
print(f"Avverage Odds Difference (AOD): {metrics_test_dir_cls.average_odds_difference():.4f}")
print(f"Differential Fairness Bias Amplification( DFBA): {metrics_test_dir_cls.differential_fairness_bias_amplification():.4f}")

In [ ]:
# Comparison of Performance Metrics
print("\nComparison of Performance Metrics Across Models:")
print(f"{'Metric':<15}{'Original':<15}{'Reweighted':<15}{'DIR Applied':<15}")
print("-" * 60)
print(f"{'Accuracy':<15}{accuracy:<15.4f}{accuracy_rw:<15.4f}{accuracy_test_dir:<15.4f}")
print(f"{'Precision':<15}{precision:<15.4f}{precision_rw:<15.4f}{precision_test_dir:<15.4f}")
print(f"{'Recall':<15}{recall:<15.4f}{recall_rw:<15.4f}{recall_test_dir:<15.4f}")
print(f"{'F1-Score':<15}{f1:<15.4f}{f1_rw:<15.4f}{f1_test_dir:<15.4f}")
print(f"{'ROC-AUC':<15}{roc_auc:<15.4f}{roc_auc_rw:<15.4f}{roc_auc_test_dir:<15.4f}")

In [ ]:
# Comparison of Fairness Metrics
print("\nComparison of Fairness Metrics Across Models:")
print(f"{'Metric':<15}{'Original':<15}{'Reweighted':<15}{'DIR Applied':<15}")
print("-" * 60)
print(f"{'SPD':<15}{metrics_test_cls.statistical_parity_difference():<15.4f}{metrics_test_rw_cls.statistical_parity_difference():<15.4f}{metrics_test_dir_cls.statistical_parity_difference():<15.4f}")
print(f"{'DI':<15}{metrics_test_cls.disparate_impact():<15.4f}{metrics_test_rw_cls.disparate_impact():<15.4f}{metrics_test_dir_cls.disparate_impact():<15.4f}")
print(f"{'SEDF':<15}{metrics_test_cls.smoothed_empirical_differential_fairness():<15.4f}{metrics_test_rw_cls.smoothed_empirical_differential_fairness():<15.4f}{metrics_test_dir_cls.smoothed_empirical_differential_fairness():<15.4f}")

Part 1: Experimentation with the DIR (assignment output should be included in 3.1 Exercise Document

Part 1.1a.

In [ ]:
# Finer granularity
levels = np.hstack([np.linspace(0., 0.1, 81), np.linspace(0.2, 1, 20)])
protected_index = train_ds.feature_names.index("Age_Group")

In [ ]:
# initialize variables to track the best repair level
di = np.array([])  # Collect Disparate Impact for all levels
train_dir_ds = None
test_dir_ds = None
X_train_dir = None
X_test_dir = None
lgb_dir_model = None

In [ ]:
# Loop through repair levels
for level in tqdm(levels):
    #apply DIR at the current repair level
    di_remover = DisparateImpactRemover(repair_level=level)
    train_dir_ds_i = di_remover.fit_transform(train_ds)
    test_dir_ds_i = di_remover.fit_transform(test_ds)

    #remove protected attribute from features
    X_train_dir_i = np.delete(train_dir_ds_i.features, protected_index, axis=1)
    X_test_dir_i = np.delete(test_dir_ds_i.features, protected_index, axis=1)

    #train LightGBM model on the repaired daataset
    lgb_dir_model_i = lgb.LGBMClassifier(
        random_state=rand, max_depth=5, num_leaves=33, verbose=-1, **lgb_params
    )
    lgb_dir_model_i.fit(X_train_dir_i, train_dir_ds_i.labels)

    # predict on the repaired dataset
    test_dir_ds_pred_i = test_dir_ds_i.copy()
    test_dir_ds_pred_i.labels = lgb_dir_model_i.predict(X_test_dir_i)

    # fairness metrics
    metrics_test_dir_ds = BinaryLabelDatasetMetric(
        test_dir_ds_pred_i,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )
    di_i = metrics_test_dir_ds.disparate_impact()

    # track and print DI for this level
    print(F"Repair Level: {level:.2f}, Disparate Impact: {di_i:.4f}")

    #update thebest results if this level is closest to DI=1
    if (di.shape[0] == 0) or (np.min(np.abs(di -1)) >= abs(di_i -1)):
        train_dir_ds = train_dir_ds_i
        test_dir_ds = test_dir_ds_i
        X_train_dir = X_train_dir_i
        X_test_dir = X_test_dir_i
        lgb_dir_model = lgb_dir_model_i

    # append to DI List
    di = np.append(di, di_i)

In [ ]:
# Plot Disparate Impact across repair levels
di = di[:len(levels)]

In [ ]:
plt.close(plt.gcf())

In [ ]:
# Use Seaborn style for better visuals
sns.set(style="whitegrid")

plt.figure(figsize=(11, 6))

In [ ]:
# Plot the Disparate Impact (DI) against repair levels
plt.plot(levels, di, marker="o", linestyle="-", color="blue", label="Disparate Impact")

In [ ]:
# Add labels, title, and legend
plt.ylabel("Disparate Impact (DI)", fontsize=14)
plt.xlabel("Repair Level", fontsize=14)
plt.title("Disparate Impact Across Repair Levels", fontsize=16)
plt.legend(fontsize=12)

In [ ]:
# Show grid for readability
plt.grid(True)

In [ ]:
# Display the plot
plt.show()

In [ ]:
from IPython.display import display
display(plt.gcf())

In [ ]:
# print the best repair level and its corresponding disparate impact
best_level = levels[np.argmin(np.abs(di - 1))]
print(f"Best Repair Level: {best_level:.2f}")
print(f"Disparate Impact at Best Level: {di[np.argmin(np.abs(di - 1))]:.4f}")

Part 1.1b.

In [ ]:
# Coarser granularity
levels = np.hstack([np.linspace(0., 0.1, 10), np.linspace(0.2, 1, 5)])
protected_index = train_ds.feature_names.index("Age_Group")

In [ ]:
# initialize variables to track the best repair level
di = np.array([])  # Collect Disparate Impact for all levels
train_dir_ds = None
test_dir_ds = None
X_train_dir = None
X_test_dir = None
lgb_dir_model = None

In [ ]:
# Loop through repair levels
for level in tqdm(levels):
    #apply DIR at the current repair level
    di_remover = DisparateImpactRemover(repair_level=level)
    train_dir_ds_i = di_remover.fit_transform(train_ds)
    test_dir_ds_i = di_remover.fit_transform(test_ds)

    #remove protected attribute from features
    X_train_dir_i = np.delete(train_dir_ds_i.features, protected_index, axis=1)
    X_test_dir_i = np.delete(test_dir_ds_i.features, protected_index, axis=1)

    #train LightGBM model on the repaired daataset
    lgb_dir_model_i = lgb.LGBMClassifier(
        random_state=rand, max_depth=5, num_leaves=33, verbose=-1, **lgb_params
    )
    lgb_dir_model_i.fit(X_train_dir_i, train_dir_ds_i.labels)

    # predict on the repaired dataset
    test_dir_ds_pred_i = test_dir_ds_i.copy()
    test_dir_ds_pred_i.labels = lgb_dir_model_i.predict(X_test_dir_i)

    # fairness metrics
    metrics_test_dir_ds = BinaryLabelDatasetMetric(
        test_dir_ds_pred_i,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )
    di_i = metrics_test_dir_ds.disparate_impact()

    # track and print DI for this level
    print(F"Repair Level: {level:.2f}, Disparate Impact: {di_i:.4f}")

    #update thebest results if this level is closest to DI=1
    if (di.shape[0] == 0) or (np.min(np.abs(di -1)) >= abs(di_i -1)):
        train_dir_ds = train_dir_ds_i
        test_dir_ds = test_dir_ds_i
        X_train_dir = X_train_dir_i
        X_test_dir = X_test_dir_i
        lgb_dir_model = lgb_dir_model_i

    # append to DI List
    di = np.append(di, di_i)

In [ ]:
# Plot Disparate Impact across repair levels
di = di[:len(levels)]

In [ ]:
plt.close(plt.gcf())

In [ ]:
# Use Seaborn style for better visuals
sns.set(style="whitegrid")

plt.figure(figsize=(11, 6))

In [ ]:
# Plot the Disparate Impact (DI) against repair levels
plt.plot(levels, di, marker="^", linestyle="-", color="red", label="Disparate Impact")

In [ ]:
# Add labels, title, and legend
plt.ylabel("Disparate Impact (DI)", fontsize=14)
plt.xlabel("Repair Level", fontsize=14)
plt.title("Disparate Impact Across Repair Levels", fontsize=16)
plt.legend(fontsize=12)

In [ ]:
# Show grid for readability
plt.grid(True)

In [ ]:
# Display the plot
plt.show()

In [ ]:
from IPython.display import display
display(plt.gcf())

In [ ]:
# print the best repair level and its corresponding disparate impact
best_level = levels[np.argmin(np.abs(di - 1))]
print(f"Best Repair Level: {best_level:.2f}")
print(f"Disparate Impact at Best Level: {di[np.argmin(np.abs(di - 1))]:.4f}")

Part 1.2): Identify the best repair level and the Disparate Impact at the best repair level.

For the original data: DI= 0.865 at Repair Level 0.5.  For fine granuality: DI % RL where the same.  For coarse granulity: DI= 0.868 @ RL 0.4.  So the
Coarse setting seemed to do best.  All inputs had the sam Zpattern.  The finer the levls, the more activity at the low repair level of 0.1.  At this
Level DI varied from 0.82 to 0.85.  For all 3 tests, with over 90% of the DI levels above 80%, bias does not seem like problem for this data set.

Part 1.3): Retrain a Light GBM model on the modified DIR data with the best repair level and
compute both the performance metrics and the fairness metrics.

In [ ]:
# train LightGbM
lgb_dir_model = LGBMClassifier(
    random_state=531,
    max_depth=6,
    num_leaves=33,
    **lgb_params
)

In [ ]:
# use reweighted dataset's instance weights
lgb_dir_model.fit(
    X_train,
    y_train,
    sample_weight=train_dir_ds.instance_weights
)

In [ ]:
# evaluate performance metrics
y_pred_test_dir = lgb_dir_model.predict(X_test)
y_pred_prob_test_dir = lgb_dir_model.predict_proba(X_test)[:, 1]

In [ ]:
# Compute Performance Metrics
accuracy_dir = accuracy_score(y_test, y_pred_test_dir)
precision_dir = precision_score(y_test, y_pred_test_dir)
recall_dir = recall_score(y_test, y_pred_test_dir)
f1_dir = f1_score(y_test, y_pred_test_dir)
roc_auc_dir = roc_auc_score(y_test, y_pred_prob_test_dir)

In [ ]:
print("\nPerformance Metrics for DIR Model:")
print(f"Accuracy: {accuracy_dir:.4f}")
print(f"Precision: {precision_dir:.4f}")
print(f"Recall: {recall_dir:.4f}")
print(f"F1-Score: {f1_dir:.4f}")
print(f"ROC-AUC: {roc_auc_dir:.4f}")

Comment: 82% accuracy is OK, Correct Positve Prediction of 65% is fair.  Recall of 54% (correct positives is poor.  
F1 00.59 shows a moderate balance between Precision and Recall.  Strong ROC at 0.8, showing model ability to distiguish
positive versus negative cases.

In [ ]:
# Compute Fairness Metrics
test_pred_dir_ds = test_ds.copy(deepcopy=True)
test_pred_dir_ds.labels = y_pred_test_dir.reshape(-1, 1)
test_pred_dir_ds.scores = y_pred_prob_test_dir.reshape(-1,1)

In [ ]:
# Fairness Metrics for the DIR Model
metrics_test_dir_cls = ClassificationMetric(
    test_ds,
    test_pred_dir_ds,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [ ]:
# Print Fairness Metrics
print("\nFairness Metrics for DIR Model:")
print(f"Statistical Parity Difference (SPD): {metrics_test_dir_cls.statistical_parity_difference():.4f}")
print(f"Disparate Impact (DI): {metrics_test_dir_cls.disparate_impact():.4f}")
print(f"Equal Opportunity Difference (EOD): {metrics_test_dir_cls.equal_opportunity_difference():.4f}")
print(f"Average Odds Difference (AOD): {metrics_test_dir_cls.average_odds_difference():.4f}")
print(f"Differential Fairness Bias Amplification (DFBA): {metrics_test_dir_cls.differential_fairness_bias_amplification():.4f}")

# Comment: SPD of -0.14 shows moderate unfairness against yhe underprivileged group.  DI of 0.84 is within acceptable range.  
EOD of -0.04 shows small bias against underpriviledged group.  AOD of -0.1 shows obvious bias against underprivileged group.
DFBA of 0.25 indicates some bias amplification.